<a href="https://colab.research.google.com/github/viyaleta/DES/blob/master/DES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
class SimLogger:
    def __init__(self, fmt='Time: %s', wheel_size=1):
        self.fmt = fmt
        self.wheel_size = wheel_size
        self.current_sim_time = tuple([0]*fmt.count('%')) 
        self.log_entries = []
        self.stats = {}
        
    def log_time(self, *argv):
        self.current_sim_time = argv
        
    def log_event(self, event):
        self.log_entries.append([self.current_sim_time, event.obj.name, 
                                 event.obj.state, event.description])
        
        if event.obj.name not in self.stats.keys():
            self.stats[event.obj.name] = {}
        if event.obj.state not in self.stats[event.obj.name].keys():
            self.stats[event.obj.name][event.obj.state] = 0
        
        
    def read(self):
        for entry in self.log_entries:
            time = self.fmt % entry[0]
            print('{time:<20} > {desc}'.format(time=time, desc=entry[-1]))
            
    def compile_stats(self):
        obj_prev_time = {obj: 0 for obj in list(set(sum(self.log_entries, [])[1::4]))}
        obj_prev_state = {obj: None for obj in list(set(sum(self.log_entries, [])[1::4]))}

        for i in range(len(self.log_entries)):
            obj = self.log_entries[i][1]
            state = self.log_entries[i][2]
            
            if len(self.log_entries[i][0])>1:
                time = self.log_entries[i][0][0]*self.wheel_size + self.log_entries[i][0][1]
            else:
                time = self.log_entries[i][0][0]
            
            prev_state = obj_prev_state[obj]
            duration = time-obj_prev_time[obj]
            if prev_state:
                self.stats[obj][prev_state] += duration

            obj_prev_time[obj] = time
            obj_prev_state[obj] = state

In [0]:
class Entity:
    def __init__(self, name, state='none', state_duration=0):
        self.name = name
        self.state = state
        self.state_duration = state_duration
    
            
class Resource(Entity):
    def __init__(self, name, in_use=False):
        Entity.__init__(self, name)
        self.in_use = in_use
        
        
class Recipient(Entity):
    def __init__(self, name, state='none', state_duration=0):
        Entity.__init__(self, name, state_duration=0)
        self.resource = None
        
        
class ServiceProvider(Entity):
    def __init__(self, name, available_states=[]):
        Entity.__init__(self, name)
        self.available_states = available_states
        self.user = None
    
    @property
    def available(self):
        return True if not self.user and self.state in self.available_states else False

In [0]:
### BASIC EVENTS ###

class Event:
    def __init__(self, obj, next_event=None):
        self.obj = obj
        self.next_event = next_event
        self.description = ''
        
    def process(self):
        return self.next_event, self.obj.state_duration
        
        
class ChangeState(Event):
    def __init__(self, obj, new_state='', new_state_duration=0, 
                 next_event=None):
        Event.__init__(self, obj, next_event)
        self.new_state = new_state
        self.new_state_duration = new_state_duration
        
    def process(self):
        self.obj.state = self.new_state
        self.obj.state_duration = self.new_state_duration
        
        self.description = '{} changed state to {}'.format(self.obj.name, self.new_state)
        return self.next_event, self.obj.state_duration

    
    
### EVENTS FOR RESOURCE ALLOCATION ###

class AllocateResource(ChangeState):
    def __init__(self, obj, new_state='', new_state_duration=0, 
                 resources=[], next_event=None):
        ChangeState.__init__(self, obj, new_state, new_state_duration, next_event)
        self.resources = resources
        
    def process(self):
        resource_dibs = None
        
        # check if the resource is in use
        for resource in self.resources:
            if resource.in_use == False:
                resource_dibs = resource
                break
                
        # if nothing, wait one more tick
        if not resource_dibs:
            self.description = '{} {} - no resources available'.format(self.obj.name, self.obj.state)
            return self, 1
            
        # change state
        old_state = self.obj.state
        self.obj.state = self.new_state
        self.obj.state_duration = self.new_state_duration
        
        # allocate the resource
        self.obj.resource = resource_dibs
        self.obj.resource.in_use = True
        
        self.description = '{} changed state from {} to {} - using {}'.format(self.obj.name, old_state, self.new_state, resource_dibs.name)
        
        return self.next_event, self.obj.state_duration
        
class DeallocateResource(ChangeState):
    def __init__(self, obj, new_state='', new_state_duration=0, next_event=None):
        ChangeState.__init__(self, obj, new_state, new_state_duration, next_event)
        
    def process(self):
        # change state
        old_state = self.obj.state
        self.obj.state = self.new_state
        self.obj.state_duration = self.new_state_duration
        
        # change resource to not in use and remove from recipient
        self.obj.resource.in_use = False
        self.obj.resource = None
        
        self.description = '{} changed state from {} to {}'.format(self.obj.name, old_state, self.obj.state)
        return self.next_event, self.obj.state_duration
    
    
### EVENTS FOR SERVICE ALLOCATION ###


class GetService(Event):
    # links state of user to a service provider
    def __init__(self, obj, service_pool):
        Event.__init__(self, obj)
        self.service_pool = service_pool
        
    def process(self):
        service_dibs = None
        
        # allocate obj from pool to obj
        for service in self.service_pool:
            if service.available:
                service_dibs = service
                break
                
        if not service_dibs:
            self.description = '{} {} - no services available'.format(self.obj.name, self.obj.state)
            return self, 1
        
        # link this object to service
        service_dibs.user = self.obj
        self.description = '{} will now service {}'.format(service_dibs.name, service_dibs.user.name)
        return None, 0
    
    
class Service(ChangeState):
    # services user using a service provider 
    def __init__(self, obj, new_state, new_state_duration, next_event=None):
        ChangeState.__init__(self, obj, new_state, new_state_duration, next_event)
        
    def process(self):        
        # waiting for a user
        if not self.obj.user:
            self.description = '{} {} - no users to service'\
                .format(self.obj.name, self.obj.state)
            return self, 1
        
        self.obj.state = self.new_state
        self.obj.state_duration = self.new_state_duration
        self.obj.user.state = self.new_state
        self.obj.user.state_duration = self.new_state_duration
        
        self.description = '{} and {} changed state to {}'\
            .format(self.obj.name, self.obj.user.name, self.new_state)
        
        return self.next_event, self.obj.state_duration
    
    
class EndService(ChangeState):
    # services user using a service provider and change state
    def __init__(self, obj, new_state, new_state_duration, next_event=None):
        ChangeState.__init__(self, obj, new_state, new_state_duration, next_event)
        
    def process(self):
        
        old_state = self.obj.state
        self.description = '{} changed state from {} to {} and {} changed state from {} to {}'\
            .format(self.obj.name, old_state, self.new_state, self.obj.user.name, old_state, 'none')
        
        self.obj.state = self.new_state
        self.obj.state_duration = self.new_state_duration
        
        self.obj.user.state = 'none'
        self.obj.user.state_duration = 0
        self.obj.user = None
        
        return self.next_event, self.obj.state_duration

In [0]:
class Stream:
    def __init__(self, until=-1):
        self.time = 0
        self.future_events = []
        self.until = until
        self.log = SimLogger('Time: %s')
        
    def run(self):
        while True:
            # progress time
            self.progress_time()

            # for all events in the current time, process event
            while len(self.future_events)>0 and self.time==self.future_events[0][0]:
                _, event = self.future_events.pop(0)
                if event:                 
                    next_event, current_event_duration = event.process()
                    self.log.log_event(event)

                    # schedule next event
                    self.schedule(next_event, current_event_duration)
                    
            if self.break_condition():
                break
                
    def break_condition(self):
        if (self.time+1)==self.until:
            return True
        
        if len(self.future_events)==0:
            return True
        
        if self.until>0 and self.future_events[0][0]>self.until:
            return True
                
    def progress_time(self):
        # grab time of the next event in the queue
        self.time = self.future_events[0][0]
        self.log.log_time(self.time)
        
    def schedule(self, event, current_event_duration):
        next_time = self.time + current_event_duration
        
        # find the correct position to insert next event (proper time, last in queue)
        for i in range(len(self.future_events)+1):
            if i==len(self.future_events) or next_time < self.future_events[i][0]:
                self.future_events.insert(i, (next_time, event))
                break
                
                
class Wheel(Stream):
    def __init__(self, ticks, max_turns=-1):
        Stream.__init__(self)
        self.ticks = ticks
        self.wheel_turn = 0
        self.max_turns = max_turns
        self.next_wheel_events = []
        self.log = SimLogger('Wheel: %s Time: %s', self.ticks)
        
    def break_condition(self):
        if (self.time+1)==self.ticks and (self.wheel_turn+1)==self.max_turns:
            return True
        
        if len(self.future_events)==0 and len(self.next_wheel_events)==0:
            return True
        
        
    def progress_time(self):            
        
        if len(self.future_events)>0 and self.time == self.future_events[0][0]:
            return  # don't progress - we still have more events to process
        
        # up time by one
        self.time += 1
        
        # if necessary, turn the wheel and replace the future events list
        if self.time == self.ticks:
            self.time = 0
            self.wheel_turn += 1
            self.future_events = self.next_wheel_events
            self.next_wheel_events = []
            
            # if there are events that have time > ticks, reschedule them
            for i in range(len(self.future_events)):
                if self.future_events[i][0]>=self.ticks:
                    event = self.future_events.pop(i)
                    self.schedule(event[1], 0)
                    
        self.log.log_time(self.wheel_turn, self.time)
        
        
    def schedule(self, event, in_time):
        next_time = self.time + in_time
        
        if next_time < self.ticks:
            # find the correct position to insert next event (proper time, last in queue)
            for i in range(len(self.future_events)+1):
                if i==len(self.future_events) or next_time < self.future_events[i][0]:
                    self.future_events.insert(i, (next_time, event))
                    break
                    
        # if next time doesn't fit in this wheel, place it into a future wheel
        else:
            next_time = next_time - self.ticks
            # find the correct position to insert next event (proper time, last in queue)
            for i in range(len(self.next_wheel_events)+1):
                if i==len(self.next_wheel_events) or next_time < self.next_wheel_events[i][0]:
                    self.next_wheel_events.insert(i, (next_time, event))
                    break

In [0]:
### CONVENIENCE FUNCTIONS ###

def link_events(events):
    for i in range(1, len(events)):
        events[i-1].next_event = events[i]
        
        
def circular_link_events(events):
    for i in range(1, len(events)):
        events[i-1].next_event = events[i]
    events[-1].next_event = events[0]

In [112]:
# simple car
car = Entity('car')

start = ChangeState(car, 'on', 1)
go = ChangeState(car, 'driving', 10)
stop = ChangeState(car, 'stopped', 1)

link_events([start, go, stop])

s = Stream()
s.schedule(start, 10)

s.run()
s.log.read()

Time: 10             > car changed state to on
Time: 11             > car changed state to driving
Time: 21             > car changed state to stopped


In [113]:
s.log.compile_stats()
s.log.stats

{'car': {'driving': 10, 'on': 1, 'stopped': 0}}

In [114]:
# simple ATM machine

customer = Entity('customer')

queue = ChangeState(customer, 'in queue', 0)
use = ChangeState(customer, 'using machine', 3)
leave = ChangeState(customer, 'left', 0)
link_events([queue, use, leave])

s = Stream()
s.schedule(queue, 0)

s.run()
s.log.read()

Time: 0              > customer changed state to in queue
Time: 0              > customer changed state to using machine
Time: 3              > customer changed state to left


In [115]:
s.log.compile_stats()
s.log.stats

{'customer': {'in queue': 0, 'left': 0, 'using machine': 3}}

In [116]:
# simple bank simulation
customer = Recipient('customer')
ATM = Resource('ATM')

queue = ChangeState(customer, 'in queue', 0)
get_money = AllocateResource(customer, 'using machine', 3, [ATM])
leave = DeallocateResource(customer, 'left', 0)
link_events([queue, get_money, leave])

s = Stream()
s.schedule(queue, 0)

s.run()
s.log.read()

Time: 0              > customer changed state to in queue
Time: 0              > customer changed state from in queue to using machine - using ATM
Time: 3              > customer changed state from using machine to left


In [117]:
s.log.compile_stats()
s.log.stats

{'customer': {'in queue': 0, 'left': 0, 'using machine': 3}}

In [118]:
# queue bank simulation
customer_1 = Recipient('customer_1')
customer_2 = Recipient('customer_2')
ATM = Resource('ATM')

customer_1_arrives = ChangeState(customer_1, 'in queue', 0)
link_events([
    customer_1_arrives,
    AllocateResource(customer_1, 'using machine', 3, [ATM]),
    DeallocateResource(customer_1, 'left', 0)
])

customer_2_arrives = ChangeState(customer_2, 'in queue', 0)
link_events([
    customer_2_arrives,
    AllocateResource(customer_2, 'using machine', 3, [ATM]),
    DeallocateResource(customer_2, 'left', 0)
])

s = Stream()
s.schedule(customer_1_arrives, 0)
s.schedule(customer_2_arrives, 1)

s.run()
s.log.read()

Time: 0              > customer_1 changed state to in queue
Time: 0              > customer_1 changed state from in queue to using machine - using ATM
Time: 1              > customer_2 changed state to in queue
Time: 1              > customer_2 in queue - no resources available
Time: 2              > customer_2 in queue - no resources available
Time: 3              > customer_1 changed state from using machine to left
Time: 3              > customer_2 changed state from in queue to using machine - using ATM
Time: 6              > customer_2 changed state from using machine to left


In [119]:
s.log.compile_stats()
s.log.stats

{'customer_1': {'in queue': 0, 'left': 0, 'using machine': 3},
 'customer_2': {'in queue': 2, 'left': 0, 'using machine': 3}}

In [120]:
# multiple resources bank simulation
customer_1 = Recipient('customer_1')
customer_2 = Recipient('customer_2')
customer_3 = Recipient('customer_3')
ATM_1 = Resource('ATM_1')
ATM_2 = Resource('ATM_2')

customer_1_arrives = ChangeState(customer_1, 'in queue', 0)
link_events([
    customer_1_arrives,
    AllocateResource(customer_1, 'using machine', 3, [ATM_1, ATM_2]),
    DeallocateResource(customer_1, 'left', 0)
])

customer_2_arrives = ChangeState(customer_2, 'in queue', 0)
link_events([
    customer_2_arrives,
    AllocateResource(customer_2, 'using machine', 3, [ATM_1, ATM_2]),
    DeallocateResource(customer_2, 'left', 0)
])

customer_3_arrives = ChangeState(customer_3, 'in queue', 0)
link_events([
    customer_3_arrives,
    AllocateResource(customer_3, 'using machine', 3, [ATM_1, ATM_2]),
    DeallocateResource(customer_3, 'left', 0)
])

s = Stream()
s.schedule(customer_1_arrives, 0)
s.schedule(customer_2_arrives, 1)
s.schedule(customer_3_arrives, 2)

s.run()
s.log.read()

Time: 0              > customer_1 changed state to in queue
Time: 0              > customer_1 changed state from in queue to using machine - using ATM_1
Time: 1              > customer_2 changed state to in queue
Time: 1              > customer_2 changed state from in queue to using machine - using ATM_2
Time: 2              > customer_3 changed state to in queue
Time: 2              > customer_3 in queue - no resources available
Time: 3              > customer_1 changed state from using machine to left
Time: 3              > customer_3 changed state from in queue to using machine - using ATM_1
Time: 4              > customer_2 changed state from using machine to left
Time: 6              > customer_3 changed state from using machine to left


In [121]:
s.log.compile_stats()
s.log.stats

{'customer_1': {'in queue': 0, 'left': 0, 'using machine': 3},
 'customer_2': {'in queue': 0, 'left': 0, 'using machine': 3},
 'customer_3': {'in queue': 1, 'left': 0, 'using machine': 3}}

In [122]:
# simple car - timing wheel
car = Entity('car')

start = ChangeState(car, 'idle', 1)

link_events([
    start, 
    ChangeState(car, 'driving', 10),
    ChangeState(car, 'waiting at light', 2),
    ChangeState(car, 'driving', 14),
    ChangeState(car, 'stopped', 10)
])

w = Wheel(16)
w.schedule(start, 10)

w.run()
w.log.read()

Wheel: 0 Time: 10    > car changed state to idle
Wheel: 0 Time: 11    > car changed state to driving
Wheel: 1 Time: 5     > car changed state to waiting at light
Wheel: 1 Time: 7     > car changed state to driving
Wheel: 2 Time: 5     > car changed state to stopped


In [123]:
w.log.compile_stats()
w.log.stats

{'car': {'driving': 24, 'idle': 1, 'stopped': 0, 'waiting at light': 2}}

In [124]:
# Airport Taxi Allocation (2 people)

person_1 = Entity('person_1')
person_2 = Entity('person_2')

taxi_1 = ServiceProvider('taxi_1', available_states=['waiting'])

person_1_intent = ChangeState(person_1, 'in queue', 0, GetService(person_1, [taxi_1]))
person_2_intent = ChangeState(person_2, 'in queue', 0, GetService(person_2, [taxi_1]))

taxi_1_waiting = ChangeState(taxi_1, 'waiting', 0)
circular_link_events([
    taxi_1_waiting,
    Service(taxi_1, 'traveling downtown', 20),
    EndService(taxi_1, 'traveling to airport', 18),
])

s = Stream(until=100)
s.schedule(person_1_intent, 1)
s.schedule(person_2_intent, 2)
s.schedule(taxi_1_waiting, 3)
s.run()
# s.log.read()

s.log.compile_stats()
s.log.stats

{'person_1': {'in queue': 2},
 'person_2': {'in queue': 39},
 'taxi_1': {'traveling downtown': 40,
  'traveling to airport': 36,
  'waiting': 20}}

In [127]:
# Airport Taxi Allocation

person_1 = Entity('person_1')
person_2 = Entity('person_2')
person_3 = Entity('person_3')
person_4 = Entity('person_4')
person_5 = Entity('person_5')
taxi_1 = ServiceProvider('taxi_1', ['waiting'])
taxi_2 = ServiceProvider('taxi_2', ['waiting'])

person_1_intent = ChangeState(person_1, 'in queue', 0, GetService(person_1, [taxi_1, taxi_2]))
person_2_intent = ChangeState(person_2, 'in queue', 0, GetService(person_2, [taxi_1, taxi_2]))
person_3_intent = ChangeState(person_3, 'in queue', 0, GetService(person_3, [taxi_1, taxi_2]))
person_4_intent = ChangeState(person_4, 'in queue', 0, GetService(person_4, [taxi_1, taxi_2]))
person_5_intent = ChangeState(person_5, 'in queue', 0, GetService(person_5, [taxi_1, taxi_2]))


taxi_1_waiting = ChangeState(taxi_1, 'waiting', 0)
circular_link_events([
    taxi_1_waiting,
    Service(taxi_1, 'traveling downtown', 20),
    EndService(taxi_1, 'traveling to airport', 18),
])

taxi_2_waiting = ChangeState(taxi_2, 'waiting', 0)
circular_link_events([
    taxi_2_waiting,
    Service(taxi_2, 'traveling downtown', 15),
    EndService(taxi_2, 'traveling to airport', 14),
])

s = Stream(until=155)
s.schedule(person_1_intent, 1)
s.schedule(person_2_intent, 4)
s.schedule(person_3_intent, 5)
s.schedule(person_4_intent, 16)
s.schedule(person_5_intent, 10)
s.schedule(taxi_1_waiting, 0)
s.schedule(taxi_2_waiting, 5)

s.run()
# s.log.read()

s.log.compile_stats()
s.log.stats

{'person_1': {'in queue': 0},
 'person_2': {'in queue': 1},
 'person_3': {'in queue': 29},
 'person_4': {'in queue': 47},
 'person_5': {'in queue': 30},
 'taxi_1': {'traveling downtown': 40,
  'traveling to airport': 36,
  'waiting': 78},
 'taxi_2': {'traveling downtown': 45,
  'traveling to airport': 42,
  'waiting': 62}}

In [128]:
# Airport Taxi Allocation with Timing Wheel (2 people)

person_1 = Entity('person_1')
person_2 = Entity('person_2')

taxi_1 = ServiceProvider('taxi_1', available_states=['waiting'])

person_1_intent = ChangeState(person_1, 'in queue', 0, GetService(person_1, [taxi_1]))
person_2_intent = ChangeState(person_2, 'in queue', 0, GetService(person_2, [taxi_1]))

taxi_1_waiting = ChangeState(taxi_1, 'waiting', 0)
circular_link_events([
    taxi_1_waiting,
    Service(taxi_1, 'traveling downtown', 20),
    EndService(taxi_1, 'traveling to airport', 18),
])

w = Wheel(20, max_turns=5)
w.schedule(person_1_intent, 1)
w.schedule(person_2_intent, 2)
w.schedule(taxi_1_waiting, 3)
w.run()
# w.log.read()


w.log.compile_stats()
w.log.stats

{'person_1': {'in queue': 2},
 'person_2': {'in queue': 39},
 'taxi_1': {'traveling downtown': 40,
  'traveling to airport': 36,
  'waiting': 20}}